Copyright 2024 Lehi Gracia

### Save?

In [1]:
save_to_mongo = False

import subprocess, os

if save_to_mongo:
    mongod = subprocess.Popen(
        ['mongod', '--dbpath', os.path.expanduser(os.getenv('MONGODB_PATH')), '--logpath', os.path.expanduser(os.getenv('MONGODB_LOG'))]
    )

### Implied Volaility

In [4]:
from yfinance_utils import file_utils
from finta import TA
from plotly.subplots import make_subplots
import plotly.graph_objects as go

_t = '^VIX'

COLUMNS = ['Adj Close', 'Open','High','Low','Close','Volume']
_d = file_utils.get_history(_t)
_d['sma'] = TA.SMA(_d, period=30)

_f=make_subplots(specs=[[{"secondary_y": True}]])
_f.update_layout(title=f"Market Volatility (VIX)", title_x=0.5, width=600, height=400)
_d.reset_index(inplace=True)
_f.add_trace(
    go.Scatter(y=_d['Close'], x=_d['Date'], name="VIX"),
    secondary_y=False
)
_f.add_trace(
    go.Scatter(y=_d['sma'], x=_d['Date'], name=f'sma'),
    secondary_y=False
)
_f.show()

###  Getting best performing stocks

In [5]:


import warnings
warnings.filterwarnings('ignore')

from yfinance_utils import list_utils, file_utils
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from finta import TA
from yfinance_utils import list_utils
import time
from datetime import datetime

today = datetime.today().strftime('%Y-%m-%d')
NUMBER_OF_TOP_PERFORMERS = 20
PERIOD = 50

_list = list_utils.get_short_list()
_top = list_utils.get_top_performers(_list, size=NUMBER_OF_TOP_PERFORMERS)
_sym = dict(_top)
for _s in _sym:
    time.sleep(5)
    data = file_utils.get_history(_s)
    _f=make_subplots(specs=[[{"secondary_y": True}]])
    _f.update_layout(title=f"TOP {NUMBER_OF_TOP_PERFORMERS}", title_x=0.5, width=600, height=400)

    data.columns = COLUMNS
    data['sma'] = TA.SMA(data, period=PERIOD)
    data.reset_index(inplace=True)
    _f.add_trace(
        go.Scatter(y=data['Close'], x=data['Date'], name=f"{_s}: {_sym[_s]} %"),
        secondary_y=False
    )
    _f.add_trace(
        go.Scatter(y=data['sma'], x=data['Date'], name=f'SMA {PERIOD}'),
        secondary_y=False
    )
    _f.update_yaxes(side='right')
    price = f"Date: {today}, Open:{round(data['Open'].iloc[-1],2)}, High:{round(data['High'].iloc[-1],2)}, Low:{round(data['Low'].iloc[-1],2)}, Close: {round(data['Close'].iloc[-1],2)}"
    _f.add_annotation(xref='x domain',
                    yref='y domain',
                    x=0.01,
                    y=1,
                    text=price, 
                    showarrow=False,
                    font=dict(size=8),
                    row=1, col=1)
    _f.update_layout(legend={"title_font_family":"Arial", "font":{"size":8}})
    _f.show()

if save_to_mongo:
    _sym['DATE'] = datetime.today().strftime('%Y-%m-%d')
    file_utils.save_to_mongo(_sym, "top_performers")

In [ ]:
print('The END')

The END
